<div class="alert alert-block alert-info">

### **Análisis de Base de Datos de Libros — Proyecto SQL**

Este proyecto tiene como objetivo analizar el comportamiento y la estructura de una plataforma digital de libros, utilizando exclusivamente consultas SQL. La base de datos incluye información sobre libros, autores, editoriales, calificaciones y reseñas escritas por los usuarios.

El propósito principal del estudio es responder a cinco preguntas clave relacionadas con publicaciones recientes, participación de los usuarios y desempeño de autores y editoriales, con el fin de generar valor para una futura propuesta de producto en el sector editorial digital.

Para lograrlo, se utilizaron consultas SQL estructuradas para obtener datos precisos directamente desde la base. Las respuestas se generaron en Python, pero exclusivamente a partir de instrucciones SQL, conforme a los criterios del proyecto.

In [61]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'], 
    db_config['pwd'], 
    db_config['host'], 
    db_config['port'], 
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

In [62]:
# Verificación de conexión – ejecutando la última consulta almacenada
pd.io.sql.read_sql(query, con = engine)

,avg_reviews_per_user
0,24.33


___

### **Consulta 1: Número de libros publicados después del 1 de enero de 2000**

**Objetivo:** Saber cuántos libros en la base de datos son “relativamente recientes”.

In [63]:
query = '''
SELECT COUNT(*) AS recent_books_count
FROM books
WHERE publication_date > '2000-01-01';
'''

pd.read_sql(query, con=engine)

,recent_books_count
0,819


<div class="alert alert-block alert-info">

**Conclusión:**  

La base de datos contiene **819 libros publicados después del 1 de enero del año 2000**, lo que representa una cantidad significativa de títulos recientes. Esta cifra puede servir como punto de partida para enfocar productos o recomendaciones hacia libros modernos y más relevantes para la audiencia actual.

### **Consulta 2: Reseñas y calificación promedio por libro**

**Objetivo:** Determinar la popularidad y percepción de cada libro mediante el número de reseñas de usuarios y su calificación promedio. Esto permitirá identificar libros que generan más interacción y mejor recepción.

In [64]:
query = '''
SELECT 
    books.book_id,
    books.title,
    COUNT(reviews.review_id) AS total_reviews,
    ROUND(AVG(ratings.rating), 2) AS average_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id, books.title
ORDER BY total_reviews DESC
LIMIT 10;
'''

pd.read_sql(query, con=engine)

,book_id,title,total_reviews,average_rating
0,948,Twilight (Twilight #1),1120,3.66
1,750,The Hobbit or There and Back Again,528,4.13
2,673,The Catcher in the Rye,516,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,480,4.29
5,75,Angels & Demons (Robert Langdon #1),420,3.68
6,301,Harry Potter and the Order of the Phoenix (Har...,375,4.19
7,779,The Lightning Thief (Percy Jackson and the Oly...,372,4.08
8,722,The Fellowship of the Ring (The Lord of the Ri...,370,4.39
9,79,Animal Farm,370,3.73


<div class="alert alert-block alert-info">

**Conclusión:**  

El libro con mayor número de reseñas es *Twilight*, con más de 1100 comentarios, aunque su calificación promedio (3.66) no es de las más altas. En contraste, títulos como *The Hobbit* o *The Fellowship of the Ring* combinan una buena cantidad de reseñas con calificaciones elevadas (4.13 y 4.39 respectivamente).

Esto indica que el volumen de interacción no siempre refleja una percepción positiva. Para destacar títulos en un producto editorial, es recomendable considerar tanto la cantidad de reseñas como la calificación promedio.

### **Consulta 3: Editorial con más libros de más de 50 páginas**

**Objetivo:** Identificar qué editorial publica mayor volumen de contenido sustancial (excluyendo folletos u obras muy breves), para enfocarse en sellos editoriales que generan libros más completos.

In [65]:
query = '''
SELECT 
    publishers.publisher,
    COUNT(books.book_id) AS book_count
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY book_count DESC
LIMIT 1;
'''

pd.read_sql(query, con=engine)

,publisher,book_count
0,Penguin Books,42


<div class="alert alert-block alert-info">

**Conclusión:**  

La editorial con mayor número de publicaciones de más de 50 páginas es **Penguin Books**, con 42 títulos. Este resultado sugiere que Penguin no solo publica una gran cantidad de libros, sino que también se enfoca en contenido sustancial.

Esto la convierte en una editorial clave para futuras alianzas, análisis de catálogo o recomendaciones dentro de productos digitales orientados a lectores frecuentes.

### **Consulta 4: Autor con la calificación promedio más alta (mínimo 50 calificaciones por libro)**

**Objetivo:** Identificar autores cuya obra sea consistentemente bien valorada por los lectores. Para asegurar fiabilidad estadística, solo se consideran libros con al menos 50 calificaciones.

In [66]:
query = '''
WITH books_with_enough_ratings AS (
    SELECT 
        book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating) >= 50
)

SELECT 
    a.author,
    ROUND(AVG(r.rating), 2) AS avg_rating
FROM ratings r
JOIN books b ON r.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
WHERE r.book_id IN (SELECT book_id FROM books_with_enough_ratings)
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1;
'''

pd.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


<div class="alert alert-block alert-info">

**Conclusión:**  

La autora con la mejor calificación promedio en libros con al menos 50 valoraciones es **J.K. Rowling/Mary GrandPré**, con un promedio de **4.29**. Esto refleja una excelente recepción de sus obras, que probablemente pertenecen a la serie de Harry Potter.

Este dato sugiere que las obras de este autor son una opción destacada para promover entre los lectores, especialmente en productos o servicios que prioricen la calidad percibida por la comunidad.

### **Consulta 5: Promedio de reseñas de texto entre usuarios que han calificado más de 50 libros**  

**Objetivo:** Determinar el nivel de compromiso de los usuarios más activos. Evaluaremos cuántas reseñas de texto escriben en promedio quienes han dejado más de 50 calificaciones, para entender mejor el comportamiento de usuarios muy participativos.

In [67]:
query = '''
WITH active_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating) > 50
)

SELECT 
    ROUND(AVG(review_count), 2) AS avg_reviews_per_user
FROM (
    SELECT 
        au.username,
        COUNT(reviews.review_id) AS review_count
    FROM active_users au
    LEFT JOIN reviews ON au.username = reviews.username
    GROUP BY au.username
) AS user_reviews;
'''

pd.read_sql(query, con=engine)

,avg_reviews_per_user
0,24.33


<div class="alert alert-block alert-info">

**Conclusión:**  

Los usuarios que han calificado más de 50 libros escriben en promedio **24.33 reseñas de texto**. Este comportamiento muestra un alto nivel de participación: no solo califican, sino que también dedican tiempo a compartir sus opiniones detalladas.

Estos usuarios representan un grupo valioso para cualquier plataforma: son influyentes, activos y probablemente apasionados por la lectura. Sería útil mantenerlos motivados con funcionalidades exclusivas, insignias o incluso acceso temprano a nuevos lanzamientos.

<div class="alert alert-block alert-info">

### **Conclusión General del Análisis SQL**

A lo largo de este análisis se llevaron a cabo cinco consultas clave que permitieron identificar tendencias y patrones en la base de datos del servicio digital de libros. Entre los hallazgos más relevantes se encuentran:

- Hay una gran cantidad de libros recientes (publicados después del año 2000), lo que sugiere una plataforma actualizada.
- “Twilight” es el libro con mayor número de reseñas, aunque no el de mayor calificación promedio.
- La editorial Penguin Books es la que más libros ha publicado con más de 50 páginas, lo que la posiciona como un actor editorial dominante.
- El autor con la mejor calificación promedio (considerando libros con más de 50 calificaciones) es **J.K. Rowling/Mary GrandPré**, destacando por su impacto y valoración.
- Los usuarios que han calificado más de 50 libros han escrito en promedio **24.33 reseñas de texto**, demostrando un fuerte nivel de compromiso y participación.

Este proyecto muestra cómo, a través del análisis SQL, es posible obtener información valiosa para apoyar decisiones estratégicas en productos digitales. El trabajo evidencia tanto habilidades técnicas como analíticas para el manejo de bases de datos reales.
